# Foundations of Artificial Intelligence and Machine Learning
## A Program by IIIT-H and TalentSprint

### To be done in Lab

The objective of this experiment is to use kNN as a kind of a recommendation/prediction for movies.

In this experiment we will use a subset of the original movielens dataset.

Consider the problem of recommending movies to users. We have M Users and N Movies. 
Now, we want to predict whether a given test user $x$ will watch movie $y$.

User $x$ has seen and not seen few movies in the past. We will use $x$'s movie watching history as a feature for our recommendation system.

We will use KNN to find the K nearest neighbour users (users with similar taste) to $x$, and make predictions based on their entries for movie $y$.

A user either had seen the movie (1) or not seen the movie (0). We can represent this as a matrix of size M×N. (M rows and N columns). We have actually used a dictionary with the keys userId and movieId to represent this matrix.

Each element of the matrix is either zero or one. If (u, m) entry in this matrix is 1, then the $u^{th}$ user has seen the movie $m$.

#### Training set
M×N binary matrix indicating seen/not-seen.
#### Test set: 
L test cases with $(x, y)$ pairs. $x$ is N-dimensional binary vector with missing $y^{th}$ entry - which we want to predict.


### Data Source

* AIML_DS_MOVIE-TRAIN_SMALLSUBSETOFMOVIELENSDATASET.csv

* AIML_DS_MOVIE-TEST_SMALLSUBSETOFMOVIELENSDATASET.csv

These have been taken (and modified) from:
http://kevinmolloy.info/teaching/cs504_2017Fall/

This is a small subset of the original movielens dataset.
https://grouplens.org/datasets/movielens/



We will use KNN to find the K nearest neighbour users (users with similar taste) to $x$, and make predictions based on their entries for the movie $y$.

We have given the code for Cosine distance, when computing nearest neighbours.

In [1]:
# Importing required packages
import pandas as pd

In [2]:
## Setting up the files

Train_set = "../../data/Datasets/AIML_DS_MOVIE-TRAIN_SMALLSUBSETOFMOVIELENSDATASET.csv"
Test_set = "../../data/Datasets/AIML_DS_MOVIE-TEST_SMALLSUBSETOFMOVIELENSDATASET.csv"

In [3]:
## Loading the data from set up files
rated = pd.read_csv(Train_set, converters={"userId":int, "movieId":int})
rated.describe()

,userId,movieId,rating
count,80045.000000,80045.000000,80045.000000
mean,345.401574,1654.711850,3.544594
std,195.180637,1887.186635,1.058349
min,0.000000,0.000000,0.500000
25%,179.000000,327.000000,3.000000
50%,363.000000,870.000000,4.000000
75%,518.000000,2337.000000,4.000000
max,670.000000,9065.000000,5.000000


In [4]:
rated

,userId,movieId,rating
0,0,0,2.5
1,0,1,3.0
2,0,2,3.0
3,0,3,2.0
4,0,5,2.0
5,0,6,2.0
6,0,7,2.0
7,0,8,3.5
8,0,10,2.5
9,0,11,1.0


In [5]:
userCount = max(rated.userId)
movieCount = max(rated.movieId)

In [6]:
userCount

670

In [7]:
movieCount

9065

In [8]:
rated.values

array([[0.000e+00, 0.000e+00, 2.500e+00],
       [0.000e+00, 1.000e+00, 3.000e+00],
       [0.000e+00, 2.000e+00, 3.000e+00],
       ...,
       [6.700e+02, 1.329e+03, 4.000e+00],
       [6.700e+02, 1.331e+03, 2.500e+00],
       [6.700e+02, 2.946e+03, 3.500e+00]])

In [9]:
seen = {}
for x in rated.values:
    seen[(int(x[0]), int(x[1]))] = 1

In [10]:
seen[(0,0)] # seen[(user_id, movie_id)]

1

In [11]:
# By this we assume user_id and movie_id is continious 
allUsersMovies = [(u,m) for u in range(userCount) for m in range(movieCount)]

In [12]:
for x in allUsersMovies:
    if x not in seen:
        seen[x] = 0

Now we have the data loaded into a dictionary, let us recast the distance function to use it. Given two users, $u_1$ and $u_2$, for a movie $mx$, we must ignore the entries for $mx$ while computing the distance

In [13]:
# This is actually the cosine distance
def distance(u1, u2, mx):
    d = 0 - seen[(u1, mx)] * seen[(u2, mx)]
    for m in range(movieCount):
        d += seen[(u1, m)] * seen[(u2, m)]
    return d

def kNN(k, givenUser, givenMovie):
    distances = []
    for u in range(userCount):
        if u != givenUser:
            distances.append([distance(u, givenUser, givenMovie), u])
    distances.sort()
    distances.reverse() ## Because cosine distances mean higher = closer
    return distances[:k] 

def prediction(k, givenUser, givenMovie):
    neighbours = kNN(k, givenUser, givenMovie)
    howmanySaw = sum([seen[(u, givenMovie)] for d, u in neighbours])
    return 2 * howmanySaw > k      ### predict 1 if more than half of the similar users have seen this movie, otherwise 0.
        

### Exercise 1

Verify the above code and check if it works

In [14]:
# Your Answer Here
prediction(5,0,9)

False

In [15]:
kNN(5,0,9)

[[11, 467], [10, 101], [9, 563], [9, 72], [8, 574]]

In [16]:
test_data = pd.read_csv(Test_set, converters={"userId":int, "movieId":int})

In [17]:
test_data[:1]

,userId,movieId,rating
0,0,4,4.0


In [18]:
# adding a column watch if rating for a particular movie is gte 4( 4 because when we describe on train data we get 50%)
test_datta['watch'] = test_data.apply(lambda row : row.rating >=4 ,axis =1)

In [38]:
# TODO : write better looping 
def accuracy(predicted_res, actual_res):
    res = 0
    for index in range(len(predicted_res)) :
        if predicted_res[index] == actual_res[index] :
            res +=1
    return (res/(len(predicted_res)))*100

In [ ]:
predicted_arr = []
k = 4
for data in test_data.values :
    print(data)
    predicted_arr.append(prediction(k,data[0],data[1]))

[0 4 4.0 True]
[0 9 2.0 False]
[0 13 4.0 True]
[0 16 2.0 False]
[0 17 2.5 False]
[1 21 5.0 True]
[1 30 4.0 True]
[1 44 5.0 True]
[1 45 5.0 True]
[1 49 4.0 True]
[1 63 3.0 False]
[1 66 2.0 False]
[1 70 4.0 True]
[1 72 4.0 True]
[1 75 4.0 True]
[1 80 4.0 True]
[1 86 3.0 False]
[1 89 5.0 True]
[1 91 5.0 True]
[1 93 3.0 False]
[2 96 3.0 False]
[2 49 4.5 True]
[2 101 2.0 False]
[2 105 5.0 True]
[2 113 4.0 True]
[2 120 4.0 True]
[2 123 3.0 False]
[2 124 4.0 True]
[2 135 4.5 True]
[3 140 5.0 True]
[3 142 3.0 False]
[3 145 3.0 False]
[3 58 5.0 True]
[3 59 5.0 True]
[3 146 3.0 False]
[3 90 3.0 False]
[3 93 5.0 True]
[3 162 4.0 True]
[3 163 4.0 True]
[3 164 5.0 True]
[3 168 5.0 True]
[3 173 5.0 True]
[3 174 5.0 True]
[3 105 5.0 True]
[3 190 5.0 True]
[3 193 5.0 True]
[3 198 5.0 True]
[3 203 4.0 True]
[3 208 4.0 True]
[3 212 3.0 False]
[3 214 5.0 True]
[3 216 5.0 True]
[3 221 5.0 True]
[3 225 5.0 True]
[3 227 5.0 True]
[3 229 5.0 True]
[3 232 5.0 True]
[3 240 3.0 False]
[3 13 4.0 True]
[3 253 5.0

[14 1670 1.5 False]
[14 1672 1.5 False]
[14 1678 4.0 True]
[14 1684 4.5 True]
[14 569 2.5 False]
[14 1695 5.0 True]
[14 571 3.0 False]
[14 1713 4.5 True]
[14 1718 2.5 False]
[14 638 1.0 False]
[14 1725 1.0 False]
[14 1729 1.5 False]
[14 1732 1.0 False]
[14 1745 2.5 False]
[14 1775 1.5 False]
[14 1778 2.5 False]
[14 1779 0.5 False]
[14 1783 0.5 False]
[14 1803 2.0 False]
[14 1804 0.5 False]
[14 1817 0.5 False]
[14 1821 1.0 False]
[14 1827 4.5 True]
[14 1831 3.5 False]
[14 1834 4.0 True]
[14 1835 1.0 False]
[14 1853 4.5 True]
[14 1854 1.0 False]
[14 1859 1.5 False]
[14 1868 3.0 False]
[14 1869 0.5 False]
[14 1881 0.5 False]
[14 1885 3.5 False]
[14 1888 1.0 False]
[14 1891 3.0 False]
[14 1894 1.5 False]
[14 1896 3.5 False]
[14 1904 4.0 True]
[14 1913 3.5 False]
[14 1915 3.0 False]
[14 1919 2.0 False]
[14 1921 3.0 False]
[15 99 4.0 True]
[15 1941 4.0 True]
[15 835 5.0 True]
[15 516 3.5 False]
[15 350 3.5 False]
[15 1942 4.5 True]
[15 1310 4.0 True]
[16 447 4.5 True]
[16 99 5.0 True]
[16 14

[22 1379 4.0 True]
[22 2436 4.0 True]
[22 2038 3.0 False]
[22 132 3.5 False]
[22 494 4.5 True]
[22 1431 3.5 False]
[22 1436 4.5 True]
[22 2451 4.0 True]
[22 375 4.5 True]
[22 377 5.0 True]
[22 497 3.5 False]
[22 1468 3.5 False]
[22 2462 4.5 True]
[22 2467 4.0 True]
[22 381 3.0 False]
[22 384 3.5 False]
[23 32 4.0 True]
[23 423 3.0 False]
[23 69 4.0 True]
[24 208 3.0 False]
[24 805 2.0 False]
[25 89 4.0 True]
[25 506 3.0 False]
[25 179 3.5 False]
[25 818 4.0 True]
[25 846 3.0 False]
[25 464 4.0 True]
[25 923 5.0 True]
[25 122 4.0 True]
[25 467 3.0 False]
[25 468 2.5 False]
[25 472 4.0 True]
[25 1126 3.0 False]
[25 1221 4.0 True]
[25 359 3.0 False]
[25 1288 2.0 False]
[25 1300 2.5 False]
[25 486 2.0 False]
[25 1347 3.5 False]
[25 130 3.5 False]
[25 415 2.0 False]
[25 2055 2.0 False]
[25 495 4.0 True]
[25 2060 3.0 False]
[25 1471 3.5 False]
[25 1478 5.0 True]
[25 1482 3.5 False]
[25 2486 4.0 True]
[25 1511 3.5 False]
[25 1519 4.0 True]
[25 2488 4.5 True]
[25 2489 2.5 False]
[25 1532 4.0 T

[40 2989 4.0 True]
[40 402 5.0 True]
[40 935 3.5 False]
[40 2330 3.5 False]
[40 3073 4.0 True]
[40 611 3.5 False]
[40 1167 4.0 True]
[40 1192 4.0 True]
[40 1241 4.0 True]
[40 1996 3.0 False]
[40 1328 4.0 True]
[40 368 3.5 False]
[40 3093 4.0 True]
[41 27 4.0 True]
[41 32 3.5 False]
[41 99 5.0 True]
[41 55 4.0 True]
[41 79 4.5 True]
[41 88 4.0 True]
[41 89 5.0 True]
[41 824 3.0 False]
[41 470 2.5 False]
[41 1435 4.0 True]
[41 134 4.0 True]
[41 3105 4.5 True]
[41 1679 4.0 True]
[41 1696 4.0 True]
[41 1836 3.5 False]
[41 1865 4.5 True]
[42 547 2.0 False]
[42 27 5.0 True]
[42 38 3.0 False]
[42 321 3.0 False]
[42 75 3.0 False]
[42 79 5.0 True]
[42 85 2.0 False]
[42 89 3.0 False]
[42 106 3.0 False]
[42 217 2.0 False]
[42 2571 1.0 False]
[42 861 3.0 False]
[42 227 3.0 False]
[42 229 2.0 False]
[42 3111 1.0 False]
[42 244 4.0 True]
[42 2512 4.0 True]
[42 283 3.0 False]
[42 407 3.0 False]
[42 344 4.0 True]
[42 315 3.0 False]
[42 1018 2.0 False]
[42 124 4.0 True]
[42 3002 2.0 False]
[42 349 2.0 

[61 143 4.0 True]
[61 179 5.0 True]
[61 1605 3.5 False]
[61 2498 3.0 False]
[61 576 4.0 True]
[61 1830 3.0 False]
[61 1846 3.5 False]
[61 3580 1.5 False]
[61 1868 3.5 False]
[62 417 5.0 True]
[62 655 5.0 True]
[62 72 4.5 True]
[62 91 5.0 True]
[62 92 4.5 True]
[62 95 4.0 True]
[62 728 2.5 False]
[62 3582 1.5 False]
[62 211 3.5 False]
[62 828 4.5 True]
[62 289 5.0 True]
[62 122 4.5 True]
[62 344 5.0 True]
[62 346 2.0 False]
[62 598 4.5 True]
[62 2267 2.5 False]
[62 369 5.0 True]
[63 27 4.0 True]
[63 148 4.0 True]
[64 157 5.0 True]
[64 734 5.0 True]
[64 106 2.0 False]
[64 198 5.0 True]
[64 2193 5.0 True]
[64 1229 5.0 True]
[65 27 4.0 True]
[65 57 4.0 True]
[65 72 5.0 True]
[65 440 3.0 False]
[65 110 3.0 False]
[65 458 4.0 True]
[65 971 4.0 True]
[66 2063 4.0 True]
[66 2069 5.0 True]
[66 3592 5.0 True]
[66 49 5.0 True]
[66 51 5.0 True]
[66 56 3.0 False]
[66 148 2.0 False]
[66 2135 5.0 True]
[66 78 5.0 True]
[66 153 4.0 True]
[66 156 4.0 True]
[66 453 3.0 False]
[66 3597 3.0 False]
[66 325

...

[104 673 4.0 True]
[104 33 2.0 False]
[104 36 2.5 False]
[104 143 3.0 False]
[104 46 3.5 False]
[104 4223 2.5 False]
[104 387 4.0 True]
[104 54 3.0 False]
[104 55 3.0 False]
[104 60 3.5 False]
[104 61 3.0 False]
[104 65 4.0 True]
[104 4229 3.0 False]
[104 71 3.5 False]
[104 89 3.5 False]
[104 90 3.0 False]
[104 2075 4.0 True]
[104 3335 2.5 False]
[104 721 3.5 False]
[104 435 3.5 False]
[104 527 3.0 False]
[104 759 3.5 False]
[104 763 4.0 True]
[104 764 4.0 True]
[104 2290 4.0 True]
[104 184 4.5 True]
[104 2185 4.0 True]
[104 191 4.0 True]
[104 440 3.0 False]
[104 198 4.0 True]
[104 202 3.0 False]
[104 2292 4.0 True]
[104 3016 2.5 False]
[104 805 3.5 False]
[104 2293 3.0 False]
[104 4475 3.5 False]
[104 824 3.5 False]
[104 827 3.0 False]
[104 4476 3.0 False]
[104 860 2.0 False]
[104 12 3.5 False]
[104 2578 4.0 True]
[104 229 3.5 False]
[104 878 2.5 False]
[104 2588 2.0 False]
[104 890 3.0 False]
[104 263 4.5 True]
[104 2601 3.0 False]
[104 897 4.0 True]
[104 464 4.0 True]
[104 2613 3.5 

[118 3120 3.0 False]
[118 3074 4.0 True]
[118 1044 4.0 True]
[118 2750 3.0 False]
[118 2751 4.0 True]
[118 3690 4.0 True]
[118 1095 3.0 False]
[118 4627 4.0 True]
[118 648 3.0 False]
[119 417 2.5 False]
[119 146 3.5 False]
[119 69 3.5 False]
[119 87 3.5 False]
[119 102 3.0 False]
[119 180 3.5 False]
[119 9 3.5 False]
[119 2204 1.5 False]
[119 817 3.0 False]
[119 110 4.0 True]
[119 826 3.0 False]
[119 111 3.5 False]
[119 229 3.5 False]
[119 901 3.0 False]
[119 270 3.5 False]
[119 2257 4.0 True]
[119 271 4.5 True]
[119 915 3.0 False]
[119 401 4.5 True]
[119 279 3.5 False]
[119 119 4.5 True]
[119 121 3.5 False]
[119 2712 3.5 False]
[119 1041 2.0 False]
[119 1054 5.0 True]
[119 19 2.5 False]
[119 1138 4.0 True]
[119 2774 3.0 False]
[119 132 4.0 True]
[120 662 4.0 True]
[120 27 5.0 True]
[120 32 3.0 False]
[120 2106 4.0 True]
[120 49 5.0 True]
[120 3333 5.0 True]
[120 77 4.0 True]
[120 88 4.0 True]
[120 90 4.0 True]
[120 432 5.0 True]
[121 29 3.0 False]
[121 31 3.0 False]
[121 43 3.0 False]

[136 90 4.0 True]
[136 2153 4.0 True]
[136 3063 2.0 False]
[136 195 5.0 True]
[136 202 5.0 True]
[136 878 5.0 True]
[136 2210 4.0 True]
[136 240 2.0 False]
[136 287 3.0 False]
[136 298 4.0 True]
[136 311 4.0 True]
[136 1229 4.0 True]
[137 99 4.5 True]
[137 72 2.5 False]
[137 79 4.0 True]
[137 157 5.0 True]
[137 327 4.0 True]
[137 464 3.0 False]
[137 598 2.5 False]
[137 477 0.5 False]
[137 364 3.0 False]
[137 365 4.0 True]
[137 128 1.0 False]
[137 137 4.0 True]
[137 3521 3.5 False]
[138 99 3.5 False]
[138 424 4.0 True]
[138 696 4.5 True]
[138 550 2.5 False]
[138 194 4.5 True]
[138 512 5.0 True]
[138 251 2.0 False]
[138 263 3.0 False]
[138 267 5.0 True]
[138 463 5.0 True]
[138 1028 4.5 True]
[138 4751 3.0 False]
[138 359 4.0 True]
[138 483 4.5 True]
[138 486 4.0 True]
[138 2410 5.0 True]
[139 3009 3.5 False]
[139 98 3.5 False]
[139 59 4.0 True]
[139 1655 3.0 False]
[140 395 2.0 False]
[140 113 5.0 True]
[140 402 4.0 True]
[140 2731 2.0 False]
[140 647 3.0 False]
[140 347 4.0 True]
[140 3

[156 2485 4.5 True]
[156 559 4.0 True]
[156 3888 4.0 True]
[156 1526 2.5 False]
[156 1536 3.5 False]
[156 3211 4.0 True]
[156 3213 3.0 False]
[156 1550 3.0 False]
[156 1551 4.0 True]
[156 1558 3.5 False]
[156 3223 4.5 True]
[156 633 4.5 True]
[156 1582 3.5 False]
[156 3238 4.0 True]
[156 1600 4.0 True]
[156 1601 4.0 True]
[156 1621 3.0 False]
[156 1648 4.5 True]
[156 4845 4.0 True]
[156 4779 4.0 True]
[156 4003 4.0 True]
[156 3296 4.0 True]
[156 1723 4.0 True]
[156 1731 3.5 False]
[156 3305 4.0 True]
[156 1776 3.5 False]
[157 53 4.0 True]
[157 215 2.5 False]
[157 2638 3.0 False]
[157 1452 4.5 True]
[158 23 3.5 False]
[158 24 4.5 True]
[158 27 4.0 True]
[158 321 1.5 False]
[158 143 5.0 True]
[158 145 2.5 False]
[158 323 2.0 False]
[158 60 1.5 False]
[158 64 3.0 False]
[158 148 0.5 False]
[158 69 3.0 False]
[158 89 3.5 False]
[158 432 3.5 False]
[158 180 5.0 True]
[158 183 4.5 True]
[158 184 4.0 True]
[158 772 4.0 True]
[158 4112 3.0 False]
[158 2208 3.5 False]
[158 2578 4.0 True]
[158 3

[176 835 5.0 True]
[176 841 5.0 True]
[176 334 4.0 True]
[176 240 3.0 False]
[176 253 5.0 True]
[176 4297 3.0 False]
[176 886 5.0 True]
[176 338 5.0 True]
[176 318 4.0 True]
[177 27 3.5 False]
[177 387 3.5 False]
[177 83 3.5 False]
[177 157 4.0 True]
[177 165 3.0 False]
[177 174 2.5 False]
[177 755 4.0 True]
[177 187 5.0 True]
[177 778 3.5 False]
[177 198 4.0 True]
[177 201 4.5 True]
[177 441 2.5 False]
[177 2297 2.5 False]
[177 229 3.5 False]
[177 118 3.0 False]
[177 4433 3.5 False]
[177 988 2.5 False]
[177 468 3.0 False]
[177 19 4.0 True]
[177 2447 3.5 False]
[177 1464 3.5 False]
[177 1496 3.0 False]
[177 1516 3.0 False]
[177 1530 4.5 True]
[177 1535 3.0 False]
[177 1714 3.0 False]
[177 3306 2.0 False]
[177 1757 3.5 False]
[178 105 5.0 True]
[178 1397 4.5 True]
[178 373 0.5 False]
[178 1496 4.5 True]
[178 1500 3.5 False]
[178 635 5.0 True]
[179 2275 4.0 True]
[179 5016 3.0 False]
[179 191 4.0 True]
[180 146 5.0 True]
[180 216 4.5 True]
[180 826 4.0 True]
[180 231 3.5 False]
[180 347 

In [45]:
accuracy(predicted_arr, test_data.watch)

100.0

### Exercise 2 

Change the distance function to compute Euclidean, and see if the prediction changes. Remember to modify the kNN function to pick the smallest distances: do not reverse()!

In [ ]:
## Your Code Here

### Exercise 3

Change the distance function to compute Manhattan, and see if the prediction changes. Remember to modify the kNN function to pick the smallest distances: do not reverse()!

In [ ]:
## Your Code Here

### Summary

In above experiment we have learnt how to build recommendation systems using KNN classifier.